In [1]:
import os
from mlu_tools.validation import validate_file
from sklearn.model_selection import train_test_split
import mlu_tools.utils as mlutils
import shutil

2025-01-31 16:22:14.046900: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-31 16:22:14.051758: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-31 16:22:14.115110: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-31 16:22:15.382741: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
collected_data_dir = "dataset/collected_data"
os.path.exists(collected_data_dir)

True

In [3]:
def get_filepath_list(directory, for_class, type="vid"):
    filepath_list = []
    for root, dirs, files in os.walk(directory):
        if files and os.path.basename(root) == for_class:
            for file in files:
                filepath = os.path.join(root, file)
                if validate_file(filepath, raise_exception=False, valid_types=[type]):
                    filepath_list.append(filepath)
    return filepath_list

In [4]:
filepath_list = get_filepath_list(collected_data_dir, for_class="0")

In [5]:
filepath_list

['collected_data/0/20250131_161659.mp4',
 'collected_data/0/20250131_161327.mp4',
 'collected_data/0/20250131_161510.mp4',
 'collected_data/0/20250131_161433.mp4',
 'collected_data/0/20250131_161250.mp4',
 'collected_data/0/20250131_161622.mp4']

In [6]:
len(filepath_list)

6

In [7]:
collected_dataset = {"train": [], "val": [], "test": []}
for i in range(3):
    train_set = []
    val_set = []
    test_set = []
    filepath_list = get_filepath_list(collected_data_dir, for_class=str(i))
    X_train, X_test = train_test_split(filepath_list, random_state=42, test_size=.3)
    X_val, X_test = train_test_split(X_test, random_state=42, test_size=.5)
    collected_dataset["train"].append(X_train)
    collected_dataset["val"].append(X_val)
    collected_dataset["test"].append(X_test)

In [8]:
collected_dataset

{'train': [['collected_data/0/20250131_161622.mp4',
   'collected_data/0/20250131_161510.mp4',
   'collected_data/0/20250131_161250.mp4',
   'collected_data/0/20250131_161433.mp4'],
  ['collected_data/1/20250131_161711.mp4',
   'collected_data/1/20250131_161523.mp4',
   'collected_data/1/20250131_161240.mp4',
   'collected_data/1/20250131_161423.mp4'],
  ['collected_data/2/20250131_161458.mp4',
   'collected_data/2/20250131_161445.mp4',
   'collected_data/2/20250131_161634.mp4',
   'collected_data/2/20250131_161647.mp4']],
 'val': [['collected_data/0/20250131_161659.mp4'],
  ['collected_data/1/20250131_161340.mp4'],
  ['collected_data/2/20250131_161303.mp4']],
 'test': [['collected_data/0/20250131_161327.mp4'],
  ['collected_data/1/20250131_161612.mp4'],
  ['collected_data/2/20250131_161315.mp4']]}

In [ ]:
# Downloading previous dataset
previous_dataset_url = "https://mega.nz/file/O2hxiThK#7-j01wxR6tDsTkO5YtWjUQH_QOTiOHP3QK7WIuxCEP0"
previous_dataset_path = "dataset/dataset_merged.zip"
mlutils.download(previous_dataset_url, previous_dataset_path, download_from="mega", force=True)

100%|██████████| 372M/372M [04:55<00:00, 1.26MiB/s] 

File downloaded: dataset_merged.zip


In [10]:
mlutils.unpack_archive(previous_dataset_path)

Archive unpacked to ./dataset_merged


In [11]:
previous_dataset_extracted_path = "./dataset/dataset_merged"

In [ ]:
merged_dataset_dir = "dataset/dataset_merged2"
shutil.copytree(previous_dataset_extracted_path, merged_dataset_dir)

'dataset_merged2'

In [13]:
mlutils.tree(previous_dataset_extracted_path)

└── dataset_merged
    └── videos
        ├── test
        │   ├── 0 - 8
        │   ├── 1 - 9
        │   └── 2 - 8
        ├── train
        │   ├── 0 - 20
        │   ├── 1 - 19
        │   └── 2 - 20
        └── val
            ├── 0 - 9
            ├── 1 - 9
            └── 2 - 9


In [14]:
mlutils.tree(collected_data_dir)

└── collected_data
    ├── 0 - 6
    ├── 1 - 6
    └── 2 - 6


In [15]:
for split_name in collected_dataset:  # split_name is one of the following string: "train", "val", "test"
    for class_label, filepaths in enumerate(collected_dataset[split_name]):
        for filepath in filepaths:
            filename = os.path.basename(filepath)
            shutil.copy(
                filepath, 
                os.path.join(merged_dataset_dir, "videos", split_name, str(class_label))
            )

In [16]:
mlutils.tree(merged_dataset_dir)

└── dataset_merged2
    └── videos
        ├── test
        │   ├── 0 - 9
        │   ├── 1 - 10
        │   └── 2 - 9
        ├── train
        │   ├── 0 - 24
        │   ├── 1 - 23
        │   └── 2 - 24
        └── val
            ├── 0 - 10
            ├── 1 - 10
            └── 2 - 10


In [17]:
!zip -r "{merged_dataset_dir}.zip" "{merged_dataset_dir}"

  adding: dataset_merged2/ (stored 0%)
  adding: dataset_merged2/videos/ (stored 0%)
  adding: dataset_merged2/videos/val/ (stored 0%)
  adding: dataset_merged2/videos/val/0/ (stored 0%)
  adding: dataset_merged2/videos/val/0/HandWash_024_A_12_G_05.mp4 (deflated 0%)
  adding: dataset_merged2/videos/val/0/20250131_161659.mp4 (deflated 1%)
  adding: dataset_merged2/videos/val/0/20250125_144612.mp4 (deflated 1%)
  adding: dataset_merged2/videos/val/0/HandWash_022_A_11_G_05.mp4 (deflated 0%)
  adding: dataset_merged2/videos/val/0/20250125_141054.mp4 (deflated 1%)
  adding: dataset_merged2/videos/val/0/HandWash_008_A_11_G_01.mp4 (deflated 0%)
  adding: dataset_merged2/videos/val/0/HandWash_014_A_11_G_03.mp4 (deflated 0%)
  adding: dataset_merged2/videos/val/0/HandWash_011_A_12_G_02.mp4 (deflated 0%)
  adding: dataset_merged2/videos/val/0/HandWash_007_A_12_G_01.mp4 (deflated 0%)
  adding: dataset_merged2/videos/val/0/HandWash_016_A_11_G_04.mp4 (deflated 0%)
  adding: dataset_merged2/videos/v